In [12]:
# Importando as bibliotecas necessárias
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  # Para pré-processamento de dados
from sklearn.utils import shuffle  # Para embaralhar os dados
from sklearn.model_selection import train_test_split  # Para dividir o conjunto de dados em treinamento e teste
from sklearn.metrics import (  # Métricas de avaliação de modelo
    confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score, classification_report
)

from tensorflow.keras.optimizers import Adam  # Otimizador para a rede neural
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Pré-processamento de imagens

import keras  # Biblioteca Keras para criação de modelos de redes neurais
import tensorflow as tf  # TensorFlow para construção de redes neurais

import matplotlib.pyplot as plt  # Para plotagem de gráficos
import pandas as pd  # Manipulação de dados em formato tabular
import numpy as np  # Biblioteca NumPy para manipulação de matrizes
import cv2  # OpenCV para processamento de imagens
import os  # Operações com sistema de arquivos


In [13]:
path = 'Dataset2/'  # Definindo o caminho do diretório contendo o dataset
im_size = 350  # Definindo o tamanho desejado para as imagens

street_types = os.listdir(path)  # Listando os tipos de ruas disponíveis no diretório
print(street_types)  # Imprimindo os tipos de ruas encontrados
print("Tipos de ruas encontrados: ", len(street_types))  # Imprimindo o número de tipos de ruas encontrados

['clean', 'litter', 'recycle']
Tipos de ruas encontrados:  3


In [14]:
streets = []  # Lista vazia para armazenar informações das ruas

# Iterando sobre os diferentes tipos de ruas
for item in street_types:
    all_streets = os.listdir(path + '/' + item)  # Obtendo todos os nomes de arquivo na pasta
    
    # Adicionando os nomes de arquivo à lista 'streets'
    for street in all_streets:
        streets.append((item, str(path + '/' + item) + '/' + street))

# Criando um DataFrame do pandas com as informações das ruas
streets_df = pd.DataFrame(data=streets, columns=['street type', 'image'])

In [15]:
# Imprimindo o número total de ruas no conjunto de dados
print("Total number of streets in the Dataset: ", len(streets_df))

# Contando o número de ruas em cada categoria e imprimindo os resultados
street_count = streets_df['street type'].value_counts()
print("Ruas em cada categoria: ")
print(street_count)

Total number of streets in the Dataset:  5405
Ruas em cada categoria: 
street type
recycle    1875
clean      1825
litter     1705
Name: count, dtype: int64


In [16]:
images = []  # Lista para armazenar as imagens
labels = []  # Lista para armazenar as etiquetas das ruas

# Iterando sobre os diferentes tipos de ruas
for label in street_types:
    data_path = os.path.join(path, label)  # Obtendo o caminho para os dados de uma categoria
    filenames = os.listdir(data_path)  # Obtendo os nomes dos arquivos de imagem na categoria

    # Iterando sobre os arquivos de imagem na categoria
    for filename in filenames:
        img_path = os.path.join(data_path, filename)  # Obtendo o caminho completo da imagem
        img = load_img(img_path, target_size=(im_size, im_size))  # Carregando a imagem e redimensionando
        img_array = img_to_array(img)  # Convertendo a imagem para um array numpy
        images.append(img_array)  # Adicionando a imagem à lista de imagens
        labels.append(label)  # Adicionando a etiqueta da rua à lista de etiquetas

# Convertendo a lista de imagens para um array numpy e normalizando os valores de pixel
images = np.array(images)
images = images.astype('float32') / 255.0

# Obtendo a forma (shape) do array de imagens
images.shape

(5405, 350, 350, 3)

In [17]:
# Obtendo as etiquetas das ruas do DataFrame
y = streets_df['street type'].values

# Convertendo as etiquetas para valores numéricos usando LabelEncoder
y_labelenconder = LabelEncoder()
y = y_labelenconder.fit_transform(y)

# Embaralhando as imagens e suas etiquetas para garantir uma mistura aleatória
images, y = shuffle(images, y, random_state=1)

# Dividindo os dados em conjuntos de treinamento e teste
train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

# Imprimindo as formas (shapes) dos conjuntos de dados
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(5134, 350, 350, 3)
(271, 350, 350, 3)
(5134,)
(271,)


In [18]:
# Definindo o modelo da rede neural
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(im_size, im_size, 3)),  # Camada de achatamento para transformar a imagem em vetor
    keras.layers.Dense(256, activation=tf.nn.tanh),  # Camada densa com ativação tangente hiperbólica
    keras.layers.Dense(128, activation=tf.nn.relu),   # Camada densa adicional com ativação ReLU
    keras.layers.Dense(64, activation=tf.nn.relu),    # Outra camada densa adicional com ativação ReLU
    keras.layers.Dense(3, activation=tf.nn.softmax)   # Camada de saída com ativação softmax (3 classes)
])

# Compilando o modelo
model.compile(optimizer=Adam(learning_rate=0.01),
             loss='sparse_categorical_crossentropy',  # Função de perda para classificação multiclasse
             metrics=['accuracy'])  # Métrica de avaliação

# Treinando o modelo
model.fit(train_x, 
          train_y, 
          epochs=10,  # Número de épocas de treinamento
          batch_size=64,  # Tamanho do lote
          validation_data=(test_x, test_y))  # Dados de validação

# Obtendo as probabilidades de classe previstas
y_probs = model.predict(test_x)

# Obtendo as previsões finais (índices da classe com a maior probabilidade)
y_pred = np.argmax(y_probs, axis=1)

Epoch 1/10
81/81 [==============================] - 37s 413ms/step - loss: 1.1941 - accuracy: 0.3450 - val_loss: 1.0928 - val_accuracy: 0.3616
Epoch 2/10
81/81 [==============================] - 28s 342ms/step - loss: 1.1001 - accuracy: 0.3450 - val_loss: 1.1092 - val_accuracy: 0.3653
Epoch 3/10
81/81 [==============================] - 27s 333ms/step - loss: 1.1009 - accuracy: 0.3420 - val_loss: 1.0949 - val_accuracy: 0.3653
Epoch 4/10
81/81 [==============================] - 27s 329ms/step - loss: 1.1005 - accuracy: 0.3266 - val_loss: 1.0974 - val_accuracy: 0.3653
Epoch 5/10
81/81 [==============================] - 27s 328ms/step - loss: 1.0986 - accuracy: 0.3403 - val_loss: 1.0960 - val_accuracy: 0.3653
Epoch 6/10
81/81 [==============================] - 27s 332ms/step - loss: 1.0983 - accuracy: 0.3362 - val_loss: 1.0936 - val_accuracy: 0.3653
Epoch 7/10
81/81 [==============================] - 26s 324ms/step - loss: 1.0989 - accuracy: 0.3459 - val_loss: 1.0958 - val_accuracy: 0.3653

In [19]:
# Calculando a matriz de confusão
confusion = confusion_matrix(test_y, y_pred)
print("Matriz de Confusão:")
print(confusion)

# Gerando um relatório de classificação
report = classification_report(test_y, y_pred, target_names=street_types, zero_division=1)
print(report)

# Calculando a Área sob a Curva (AUC) para cada classe
for i in range(len(street_types)):
    auc = roc_auc_score(test_y == i, y_probs[:, i])
    print(f"AUC para classe {street_types[i]}: {auc}")

Matriz de Confusão:
[[ 0  0 98]
 [ 0  0 74]
 [ 0  0 99]]
              precision    recall  f1-score   support

       clean       1.00      0.00      0.00        98
      litter       1.00      0.00      0.00        74
     recycle       0.37      1.00      0.54        99

    accuracy                           0.37       271
   macro avg       0.79      0.33      0.18       271
weighted avg       0.77      0.37      0.20       271

AUC para classe clean: 0.4962545711926389
AUC para classe litter: 0.5084716696391823
AUC para classe recycle: 0.49647639182522907
